# runtime

In [ ]:
open rust_operators
open sm'_operators

()



In [ ]:
// // test

open testing
open file_system_operators

()



## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::Arg\")>]\n#endif\ntype clap_Arg = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::Command\")>]\n#endif\ntype clap_Command = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"clap::ArgMatches\")>]\n#endif\ntype clap_ArgMatches = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::Error\")>]\n#endif\ntype std_io_Error = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::Lines<$0>\")>]\n#endif\ntype std_io_Lines<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"async_std::io::BufReader<$0>\")>]\n#endif\ntype async_std_io_BufReader<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::io::BufReader<$0>\")>]\n#endif\ntype std_io_BufReader<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::Child\")>]\n#endif\ntype std_process_Child = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::ChildStderr\")>]\n#endif\ntype std_process_ChildStderr = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::ChildStdout\")>]\n#endif\ntype std_process_ChildStdout = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::Command\")>]\n#endif\ntype std_process_Command = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::Output\")>]\n#endif\ntype std_process_Output = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::process::Stdio\")>]\n#endif\ntype std_process_Stdio = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::mpsc::Receiver<$0>\")>]\n#endif\ntype std_sync_mpsc_Receiver<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::mpsc::Sender<$0>\")>]\n#endif\ntype std_sync_mpsc_Sender<'T> = class end"

()



In [ ]:
inl types () =
    rust.types ()
    sm'.types ()
    am'.types ()
    async.types ()
    seq.types ()
    threading.types ()
    types ()

()



In [ ]:
// // test

inl types () =
    file_system.types ()
    types ()

()



## os_platform

In [ ]:
nominal os_platform' = $'System.Runtime.InteropServices.OSPlatform'

union os_platform =
    | Windows

inl os_platform = function
    | Windows => $'`os_platform'.Windows' : os_platform'

()



## is_os_platform

In [ ]:
inl is_os_platform (x : os_platform') : bool =
    x |> $'System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform'

()



## is_windows

In [ ]:
inl is_windows () : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            Windows |> os_platform |> is_os_platform
        | Rust (Native) => fun () =>
            !\($'"cfg\!(windows)"')
        | _ => fun () => null ()

()



## get_executable_suffix

In [ ]:
inl get_executable_suffix () =
    if is_windows ()
    then ".exe"
    else ""

()



In [ ]:
// // test

get_executable_suffix ()

.exe

let rec method0 () : string =
    let v0 : bool option = None
    let v1 : bool = true in let mutable _v0 = v0
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = "cfg!(windows)"
    let v3 : bool = Fable.Core.RustInterop.emitRustExpr () v2
    v3
#endif
    
#if FABLE_COMPILER_RUST && WASM
    let v4 : bool = null |> unbox<bool>
    v4
#endif
    
#if FABLE_COMPILER_RUST && CONTRACT
    let v5 : bool = null |> unbox<bool>
    v5
#endif
    
#if !FABLE_COMPILER && !WASM && !CONTRACT
    let v6 : System.Runtime.InteropServices.OSPlatform = System.Runtime.InteropServices.OSPlatform.Windows
    let v7 : (System.Runtime.InteropServices.OSPlatform -> bool) = System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform
    let v8 : bool = v7 v6
    v8
#endif
    
#if FABLE_COMPILER_TYPESCRIPT
    let v9 : bool = null |> unbox<bool>
    v9
#endif
    
#if FABLE_COMPILER_PYTHON
    let v10 : bool = null |> unbox<bool>
    v10
#endif
    |> fun x -> _v0 <- Some x
    let 

## quoted

In [ ]:
union quoted =
    | Quoted
    | NotQuoted

()



## split_command

In [ ]:
union command_parse_step =
    | Start
    | Path : quoted
    | Arguments

inl split_command (command : string) =
    let rec loop (path, args) (chars : list char) step =
        match step, chars with
        | _, (('"' | '\'') :: tail) when path = "" => loop (path, args) tail (Path Quoted)
        | Path (Quoted), (('"' | '\'') :: tail) => loop (path, args) tail (Path NotQuoted)
        | Path (Quoted), (' ' :: tail) => loop ($'$"{!path} "', args) tail (Path Quoted)
        | (Start | Path _), (' ' :: tail) => loop (path, args) tail Arguments
        | Arguments, (char :: tail) => loop (path, $'$"{!args}{!char}"') tail Arguments
        | _, (char :: tail) => loop ($'$"{!path}{!char}"', args) tail step
        | _ => path |> sm'.replace @"\" "/", args
    loop (join "", "") (command |> sm'.to_char_array |> am'.to_list' |> listm'.unbox) Start

()



In [ ]:
// // test

split_command ""
|> _assert_eq ("", "")

split_command "/a/b/c"
|> _assert_eq ("/a/b/c", "")

split_command "cat file.txt"
|> _assert_eq ("cat", "file.txt")

split_command $'"""..\\..\\file.exe file1.txt file2.txt"""'
|> _assert_eq ("../../file.exe", "file1.txt file2.txt")

split_command $'\@"c:\\dir\\file.exe ""file1.txt file2.txt"""'
|> _assert_eq ("c:/dir/file.exe", $'\@"""file1.txt file2.txt"""')

split_command $'\@"""..\\..\\dir name\\file.exe"" ""file 1.txt"" file2.txt"'
|> _assert_eq ("../../dir name/file.exe", $'\@"""file 1.txt"" file2.txt"')

split_command $'\@"""..\\..\\file 1.exe"" -c \\\\""echo 1\\\\"""'
|> _assert_eq ("../../file 1.exe", $'\@"-c \\\\""echo 1\\\\"""')

split_command $'\@"..\\..\\file 1.exe -c \\\\""echo 1\\\\"""'
|> _assert_eq ("../../file", $'\@"1.exe -c \\\\""echo 1\\\\"""')

type Mut0 = {mutable l0 : int32}
and UH0 =
    | UH0_0
    | UH0_1 of char * UH0
and [<Struct>] US1 =
    | US1_0
    | US1_1
and [<Struct>] US0 =
    | US0_0
    | US0_1 of f1_0 : US1
    | US0_2
let rec method1 () : struct (string * string) =
    let v0 : string = ""
    struct (v0, v0)
and method2 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 0
    v2
and closure1 (v0 : char) (v1 : UH0) : UH0 =
    UH0_1(v0, v1)
and closure0 () (v0 : char) : (UH0 -> UH0) =
    closure1(v0)
and method3 () : (char -> (UH0 -> UH0)) =
    closure0()
and method4 (v0 : string, v1 : string, v2 : UH0, v3 : US0) : struct (string * string) =
    match v2 with
    | UH0_1(v4, v5) -> (* Cons *)
        let v6 : bool = '"' = v4
        if v6 then
            let v8 : bool = v1 = ""
            if v8 then
                let v9 : US1 = US1_0
                let v10 : US0 = US0_1(v9)
                method4(v0, v1, v5, v10)
            else
                match v3 with
                |

## process

In [ ]:
nominal process = $'System.Diagnostics.Process'
nominal process_start_info = $'System.Diagnostics.ProcessStartInfo'
nominal data_received_event_args = $'System.Diagnostics.DataReceivedEventArgs'

inl new_process (process_start_info : process_start_info) : process =
    $'new `process (StartInfo = !process_start_info)'

inl process_start (process : process) : bool =
    $'!process.Start' ()

inl process_exit_code (process : process) : i32 =
    $'!process.ExitCode'

inl process_id (process : process) : i32 =
    $'!process.Id'

inl process_has_exited (process : process) : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!process.HasExited'
        | _ => fun () => null ()

inl process_kill (process : process) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!process.Kill' ()
        | _ => fun () => null ()

inl process_begin_error_read_line (process : process) : () =
    $'!process.BeginErrorReadLine' ()

inl process_begin_output_read_line (process : process) : () =
    $'!process.BeginOutputReadLine' ()

inl process_add_output_data_received fn (process : process) : () =
    $'!process.OutputDataReceived.Add !fn '

inl process_add_error_data_received fn (process : process) : () =
    $'!process.ErrorDataReceived.Add !fn '

inl process_wait_for_exit_async (ct : threading.cancellation_token) (process : process) : async.task () =
    $'!process.WaitForExitAsync !ct '

inl event_data (e : data_received_event_args) : string =
    $'!e.Data'

()



## execution_line

In [ ]:
type execution_line =
    {
        process_id : int
        line : string
        error : bool
    }

()



## execution_options

In [ ]:
type execution_options =
    {
        command : string
        working_directory : optionm'.option' string
        cancellation_token : optionm'.option' threading.cancellation_token
        on_line : optionm'.option' (execution_line -> async.async ())
    }

inl execution_options (fn : execution_options -> execution_options) : execution_options =
    {
        command = ""
        cancellation_token = None |> optionm'.box
        on_line = None |> optionm'.box
        working_directory = None |> optionm'.box
    }
    |> fn

()



## execute_with_options_async

In [ ]:
inl execute_with_options_async (options : execution_options) : _ (i32 * string) =
    run_target function
        | Fsharp (Native) => fun () =>
            fun () =>
                inl file_name, arguments = options.command |> split_command
                inl working_directory = options.working_directory |> optionm'.unbox |> optionm'.default_value ""

                trace Debug (fun () => $'$"execute_with_options_async / options: %A{!options}"') (join _locals)

                inl utf8 = sm'.encoding_utf8 ()
                inl start_info : process_start_info = $'System.Diagnostics.ProcessStartInfo (Arguments = !arguments, StandardOutputEncoding = !utf8, WorkingDirectory = !working_directory, FileName = !file_name, CreateNoWindow = true, RedirectStandardError = true, RedirectStandardOutput = true, UseShellExecute = false)'

                inl proc = start_info |> new_process |> use
                inl output : _ string = threading.new_concurrent_stack ()

                inl event error (e : data_received_event_args) = async.new_async fun () =>
                    inl data = e |> event_data
                    if data <> null () then
                        match options.on_line |> optionm'.unbox with
                        | Some on_line =>
                            on_line
                                {
                                    process_id = proc |> process_id
                                    line = data
                                    error = error
                                }
                            |> async.do
                        | None => ()

                        trace Verbose (fun () => $'$"> {!data}"') _locals

                        inl l = if error then "[" else ""
                        inl r = if error then "]" else ""
                        output |> threading.concurrent_stack_push $'$"{!l}{!data}{!r}"'

                proc |> process_add_output_data_received (event false >> async.start_immediate)
                proc |> process_add_error_data_received (event true >> async.start_immediate)

                if proc |> process_start |> not
                then failwith $'$"execute_with_options_async / process_start error"'

                proc |> process_begin_error_read_line
                proc |> process_begin_output_read_line

                inl ct =
                    options.cancellation_token
                    |> optionm'.unbox
                    |> optionm'.default_with threading.token_none
                    |> async.merge_cancellation_token_with_default_async
                    |> async.let'

                ct |> threading.token_register fun () =>
                    if proc |> process_has_exited |> not
                    then proc |> process_kill
                |> use
                |> ignore

                inl exit_code : i32 =
                    fun () =>
                        try_unit
                            fun () =>
                                proc
                                |> process_wait_for_exit_async ct
                                |> async.await_task
                                |> async.do
                                proc |> process_exit_code |> return
                            fun ex =>
                                // with :? System.Threading.Tasks.TaskCanceledException as ex =>
                                inl ex' = ex |> sm'.format_exception
                                output |> threading.concurrent_stack_push ex'
                                inl ex : async.task_canceled_exception = ex |> unbox
                                trace Warning (fun () => $'$"execute_with_options_async / WaitForExitAsync / ex: %A{!ex}"') (join _locals)
                                (limit.min : i32) |> return
                    |> async.new_async_unit
                    |> async.let'

                inl output =
                    output
                    |> seq.rev''
                    |> fun x => x : seq.seq' string
                    |> sm'.concat "\n"

                trace Debug (fun () =>
                    $'$"execute_with_options_async / exit_code: {!exit_code} / output.Length: {!output.Length}"'
                ) (join _locals)

                (exit_code, output) |> return
            |> async.new_async_unit
        | _ => fun () =>
            global "#if FABLE_COMPILER\n[<CompilationRepresentation (CompilationRepresentationFlags.ModuleSuffix)>]\nmodule System =\n module Diagnostics =\n  type Process = unit\n  type DataReceivedEventArgs = unit\n#endif"
            null ()

()



## execute_async

In [ ]:
inl execute_async command =
    execute_with_options_async
        {
            command = command
            cancellation_token = None |> optionm'.box
            on_line = None |> optionm'.box
            working_directory = None |> optionm'.box
        }

()



In [ ]:
// // test

types ()
inl temp_folder, disposable = file_system.create_temp_directory ()
disposable |> use |> ignore
inl file_name = "test.txt"
inl path = temp_folder </> file_name

fun () =>
    inl exit_code, result = execute_async $'\@$"pwsh -c ""Get-Content {!path}"""' |> async.let'
    exit_code |> _assert_eq 1
    result |> _assert_string_contains "not exist"

    "0" |> file_system.write_all_text_async path |> async.do

    execution_options fun x => { x with
        command = $'\@$"pwsh -c ""Get-Content {!file_name}"""'
        working_directory = Some temp_folder |> optionm'.box
    }
    |> execute_with_options_async
    |> async.return_await
|> async.new_async_unit
|> async.run_with_timeout 10000
|> function
    | Some (exit_code, output) =>
        exit_code |> _assert_eq 0i32
        output |> _assert_eq (join "0")
        true
    | _ => false
|> _assert_eq true

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>]

In [ ]:
// // test

file_system.types ()
inl temp_dir, disposable = file_system.create_temp_directory ()
disposable |> use |> ignore
fun () =>
    inl path = temp_dir </> "test.txt"
    "0" |> file_system.write_all_text_async path |> async.do

    inl cts = threading.new_cancellation_token_source ()
    trace Debug (fun () => "1") _locals
    inl result =
        execution_options fun x => { x with
            command = $'\@$"pwsh -c ""Get-Content {!path}"""'
            cancellation_token = cts |> threading.cancellation_source_token |> Some |> optionm'.box
        }
        |> execute_with_options_async
        |> async.start_child
        |> async.let'
    trace Debug (fun () => "2") _locals
    async.sleep 100 |> async.do
    trace Debug (fun () => "3") _locals
    cts |> threading.cancellation_source_cancel
    trace Debug (fun () => "4") _locals
    inl exit_code, output = result |> async.let'
    trace Debug (fun () => "5") _locals
    (exit_code, output) |> return
|> async.new_async_unit
|> async.run_with_timeout 10000
|> function
    | Some (exit_code, output) =>
        exit_code |> _assert_eq -2147483648i32
        output |> _assert_eq (join "System.Threading.Tasks.TaskCanceledException: A task was canceled.")
        true
    | _ => false
|> _assert_eq true

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>]

## child

In [ ]:
nominal child = $'std_process_Child'

()



## child_stderr

In [ ]:
nominal child_stderr = $"std_process_ChildStderr"

()



## child_stdout

In [ ]:
nominal child_stdout = $"std_process_ChildStdout"

()



## process_command

In [ ]:
nominal process_command = $'std_process_Command'

()



## stdio

In [ ]:
nominal stdio = $'std_process_Stdio'

()



## output

In [ ]:
nominal output = $'std_process_Output'

()



## io_error

In [ ]:
nominal io_error = $"std_io_Error"

()



## lines

In [ ]:
nominal lines t = $'std_io_Lines<`t>'

()



## buf_reader

In [ ]:
nominal buf_reader t = $'std_io_BufReader<`t>'

()



## async_buf_reader

In [ ]:
nominal async_buf_reader t = $'async_std_io_BufReader<`t>'

()



## receiver

In [ ]:
nominal receiver t = $'std_sync_mpsc_Receiver<`t>'

()



## sender

In [ ]:
nominal sender t = $'std_sync_mpsc_Sender<`t>'

()



## split_args

In [ ]:
union args_parse_step =
    | Start
    | Quoted : char
    | Escaped : u8

let split_args (args : string) : array_base string =
    let rec loop (acc, current) (chars : list char) (step, last) =
        // trace Verbose (fun () => $'$"split_args / current: %A{!current} / acc: %A{!acc} / step: %A{!step} / last: %A{!last}"') _locals
        match step, chars with
        | Start, ('"' :: tail) =>
            loop (acc, "") tail (Quoted '"', step)
        | Escaped _, ('\\' :: tail) =>
            loop (acc, current) tail (Quoted '\\', step)
        | Escaped _, ('"' :: tail) when last = Start =>
            loop (acc, current) tail (step, step)
        | Quoted ('"'), ('"' :: tail) when last =. Quoted '\\' =>
            loop (acc, current) tail (Start, step)
        | Quoted ('\\'), ('"' :: tail) when last <>. Escaped 0 =>
            loop (acc, $'$"{!current}\\\""') tail (Quoted '"', step)
        | Quoted ('\\'), ('"' :: tail) when last =. Quoted '\\' =>
            loop (acc, $'$"{!current}\\\""') tail (Quoted '"', step)
        | Quoted ('\\'), ('"' :: tail) =>
            loop (acc, current) tail (Start, step)
        | Escaped n, ('"' :: tail) =>
            loop (acc, $'$"{!current}\\\""') tail (Escaped n, step)
        | Quoted _, ('\\' :: tail) =>
            loop (acc, current) tail (Escaped 1, step)
        | Quoted _, ('"' :: tail) when last =. Escaped 0 =>
            loop (acc, $'$"{!current}\\\""') tail (Quoted '"', step)
        | Quoted _, ('"' :: tail) =>
            loop (acc /@ [ current ], "") tail (Start, step)
        | (Escaped _ | Quoted _), (' ' :: tail) =>
            loop (acc, $'$"{!current} "') tail (step, step)
        | _, ('\\' :: tail) =>
            loop (acc, current) tail (Escaped 0, step)
        | _, (' ' :: tail) =>
            loop ((if current = "" then acc else acc /@ [ current ]), "") tail (step, step)
        | Escaped (1), (char :: tail) when last <>. step =>
            loop (acc, $'$"{!current}\\{!char}"') tail (last, step)
        | _, (char :: tail) =>
            loop (acc, $'$"{!current}{!char}"') tail (step, step)
        | _ =>
            (if current = "" then acc else acc /@ [ current ]), current
    loop
        ([], "")
        (
            args
            |> optionm'.of_obj
            |> optionm'.unbox
            |> optionm'.default_value ""
            |> sm'.to_char_array
            |> am'.to_list'
            |> listm'.unbox
        )
        (Start, Start)
    |> fst
    |> listm'.box
    |> listm'.to_array'
    |> fun (a x : _ i32 _) => x

()



In [ ]:
// // test
// // print_code=false

"a b \"c d\" e"
|> split_args
|> _assert_eq' ;[ "a"; "b"; "c d"; "e" ]

[|"a"; "b"; "c d"; "e"|]


In [ ]:
// // test
// // print_code=false

"a b e"
|> split_args
|> _assert_eq' ;[ "a"; "b"; "e" ]

[|"a"; "b"; "e"|]


In [ ]:
// // test
// // print_code=false

"\"a b\" \"e\" \"f\""
|> split_args
|> _assert_eq' ;[ "a b"; "e"; "f" ]

[|"a b"; "e"; "f"|]


In [ ]:
// // test
// // print_code=false

"a -b \"c \\\"d\\\"\""
|> split_args
|> _assert_eq' ;[ "a"; "-b"; "c \"d\"" ]

[|"a"; "-b"; "c "d""|]


In [ ]:
// // test
// // print_code=false

"a -b \"c \\\"d\\\" e\""
|> split_args
|> _assert_eq' ;[ "a"; "-b"; "c \"d\" e" ]

[|"a"; "-b"; "c "d" e"|]


In [ ]:
// // test
// // print_code=false

$'$"--text \\\\\\"\'\'\' value \'\'\'\\\\\\" "'
|> split_args
|> _assert_eq' ;[ "--text"; "''' value '''" ]

[|"--text"; "''' value '''"|]


In [ ]:
// // test
// // print_code=false

$'\@$"run ""get c:\\test.txt"""'
|> split_args
|> _assert_eq' ;[ "run"; "get c:\\test.txt" ]

[|"run"; "get c:\test.txt"|]


## execute_with_options

In [ ]:
inl execute_with_options (options : execution_options) : i32 * string =
    run_target function
        | Fsharp (Native) => fun () =>
            options |> execute_with_options_async |> async.run_synchronously
        | Rust (Native) => fun () =>
            inl file_name, arguments = options.command |> split_command
            inl arguments =
                arguments
                |> split_args
                |> am'.to_vec
                |> am'.vec_map sm'.to_std_string

            trace Debug (fun () => $'$"execute_with_options / file_name: {!file_name} / arguments: %A{!arguments} / options: %A{!options}"') (join _locals)
            
            fun () =>
                inl command : process_command = !\\(file_name, $'"std::process::Command::new(&*$0)"')

                (!\($'"true; let mut !command = !command"') : bool) |> ignore
                inl command : rust.ref' (rust.mut' process_command) = !\\((command, arguments), $'"std::process::Command::args(&mut $0, &*$1)"')

                (!\($'"true; let mut !command = !command"') : bool) |> ignore
                inl command : rust.ref' (rust.mut' process_command) = !\\(command, $'"std::process::Command::stdout(&mut $0, std::process::Stdio::piped())"')

                (!\($'"true; let mut !command = !command"') : bool) |> ignore
                inl command : rust.ref' (rust.mut' process_command) = !\\(command, $'"std::process::Command::stderr(&mut $0, std::process::Stdio::piped())"')

                inl command =
                    match options.working_directory |> optionm'.unbox with
                    | Some working_directory =>
                        inl working_directory = join working_directory
                        !\($'"std::process::Command::current_dir(!command, &*!working_directory)"')
                    | None => command

                (!\($'"true; let mut !command = !command"') : bool) |> ignore
                inl command : resultm.result' child io_error = !\\(command, $'"std::process::Command::spawn(&mut $0)"')

                match command |> resultm.map' (optionm'.some' >> rust.new_arc_mutex) |> resultm.map_error' sm'.format' |> resultm.unbox with
                | Ok child =>
                    inl child = join child
                    inl stdout : child_stdout = !\($'"!child.lock().unwrap().as_mut().unwrap().stdout.take().unwrap()"')
                    inl stderr : child_stderr = !\($'"!child.lock().unwrap().as_mut().unwrap().stderr.take().unwrap()"')
                    inl stdout = stdout |> optionm'.some' |> rust.new_arc_mutex
                    inl stderr = stderr |> optionm'.some' |> rust.new_arc_mutex

                    inl channel_sender, channel_receiver : sender sm'.std_string * rust.arc (receiver sm'.std_string) =
                        !\($'"{ let (sender, receiver) = std::sync::mpsc::channel(); (sender, std::sync::Arc::new(receiver)) }"')
                    inl channel_receiver = join channel_receiver
                    (!\\(channel_receiver, $'"true; let !channel_receiver = !channel_receiver"') : bool) |> ignore

                    inl channel_sender = channel_sender |> rust.new_arc_mutex
                    inl channel_sender' = channel_sender |> rust.clone
                    inl channel_sender'' = channel_sender |> rust.clone
                    inl channel_receiver' = channel_receiver |> rust.new_arc_mutex

                    inl std_line (stdout : bool) channel_sender (line : _ _ io_error) =
                        inl line =
                            match line |> resultm.map_error' sm'.format' |> resultm.unbox with
                            | Ok (line : sm'.std_string) =>
                                trace Verbose (fun () => $'$"> {!line}"') _locals
                                line
                            | Error (e : sm'.std_string) =>
                                trace Critical (fun () => $'$"execute_with_options / stdout_line / stdout: {!stdout} / e: {!e}"') _locals
                                $'$"[{!e}]"' |> sm'.to_std_string
                        (!\\((channel_sender, line), $'"true; $0.lock().unwrap().send($1).unwrap()"') : bool) |> ignore

                    inl stdout_thread : threading.join_handle () = !\($'"std::thread::spawn(move || { //"')
                    inl stdout_lines : lines (buf_reader child_stdout) = !\($'"std::io::BufRead::lines(std::io::BufReader::new(!stdout.lock().unwrap().take().unwrap()))"')
                    inl stdout_lines = stdout_lines |> rust.new_arc_mutex
                    inl std_line' = std_line true channel_sender'
                    inl std_line' = join std_line'
                    (!\\(stdout_lines, $'"true; for line in $0.lock().unwrap().by_ref() { !std_line'(line) }"') : bool) |> ignore
                    (!\($'"true; })"') : bool) |> ignore

                    inl stderr_thread : threading.join_handle () = !\($'"std::thread::spawn(move || { //"')
                    inl stderr_lines : lines (buf_reader child_stderr) = !\($'"std::io::BufRead::lines(std::io::BufReader::new(!stderr.lock().unwrap().take().unwrap()))"')
                    inl stderr_lines = stderr_lines |> rust.new_arc_mutex
                    inl std_line' = std_line false channel_sender
                    inl std_line' = join std_line'
                    (!\\(stderr_lines, $'"true; for line in $0.lock().unwrap().by_ref() { !std_line'(line) }"') : bool) |> ignore
                    (!\($'"true; })"') : bool) |> ignore

                    inl output : resultm.result' output io_error = !\($'"!child.lock().unwrap().take().unwrap().wait_with_output()"')
                    inl output = output |> resultm.map_error' sm'.format'

                    (!\($'"true; !stdout_thread.join().unwrap()"') : bool) |> ignore
                    (!\($'"true; !stderr_thread.join().unwrap()"') : bool) |> ignore

                    match output |> resultm.unbox with
                    | Ok output =>
                        inl exit_code : i32 = !\\(output, $'"$0.status.code().unwrap()"')
                        exit_code, None, Some channel_receiver'
                    | Error error =>
                        trace Critical (fun () => $'$"execute_with_options -2 / error: {!error}"') _locals
                        -2i32, error |> Some, None
                | Error error =>
                    trace Critical (fun () => $'$"execute_with_options -1 / error: {!error}"') _locals
                    -1i32, error |> Some, None
                |> function
                    | exit_code, std_trace, channel_receiver =>
                        inl std_trace =
                            channel_receiver
                            |> optionm.map fun channel_receiver =>
                                !\\(channel_receiver, $'"$0.lock().unwrap().iter().collect::<Vec<String>>().join(\\"\\\\n\\")"')
                            |> optionm.map sm'.from_std_string
                            |> optionm'.default_value (
                                std_trace
                                |> optionm.map sm'.from_std_string
                                |> optionm'.default_value ""
                            )
                        trace Verbose (fun () => $'$"execute_with_options / exit_code: {!exit_code} / std_trace.Length: {!std_trace.Length}"') _locals
                        new_pair exit_code std_trace
            |> async.future_init (3, 2) 1
            |> async.block_on
            |> from_pair
        | _ => fun () => null ()

()



In [ ]:
inl execute command =
    execute_with_options
        {
            command = command
            cancellation_token = None |> optionm'.box
            on_line = None |> optionm'.box
            working_directory = None |> optionm'.box
        }

()



In [ ]:
// // test
// // print_code=false
// // rust=

types ()
inl temp_folder, disposable = file_system.create_temp_directory ()
fun () =>
    inl file_name = join "test.txt"
    inl path = temp_folder </> file_name
    inl exit_code, result = execute $'\@$"pwsh -c ""Get-Content {!path}"""'
    exit_code |> _assert_eq 1
    result |> _assert_string_contains "not exist"

    "0" |> file_system.write_all_text path

    execution_options fun x => { x with
        command = $'\@$"pwsh -c ""Get-Content {!file_name}"""'
        working_directory = Some temp_folder |> optionm'.box
    }
    |> execute_with_options
|> fun fn => fn () |> Some
|> function
    | Some (exit_code, output) =>
        exit_code |> _assert_eq 0i32
        output |> _assert_eq (join "0")
        true
    | _ => false
|> _assert_eq true
disposable |> use |> ignore

1


## command

In [ ]:
nominal command = $'clap_Command'

()



## new_command

In [ ]:
inl new_command (s : rust.static_ref' sm'.str) : command =
    !\\(s, $'"clap::Command::new($0)"')

()



In [ ]:
// // test
// // print_code=false
// // rust=

types ()
inl command = ##"command" |> new_command
trace Debug
    fun () => $'$"new_command"'
    fun () =>
        inl command = command |> sm'.format_pretty'
        $'$"command: {!command} / {!_locals ()}"'

11:07:09 debug   #1 new_command / command: Command {
    name: "command",
    long_flag: None,
    short_flag: None,
    display_name: None,
    bin_name: None,
    author: None,
    version: None,
    long_version: None,
    about: None,
    long_about: None,
    before_help: None,
    before_long_help: None,
    after_help: None,
    after_long_help: None,
    aliases: [],
    short_flag_aliases: [],
    long_flag_aliases: [],
    usage_str: None,
    usage_name: None,
    help_str: None,
    disp_ord: None,
    template: None,
    settings: AppFlags(
        0,
    ),
    g_settings: AppFlags(
        0,
    ),
    args: MKeyMap {
        args: [],
        keys: [],
    },
    subcommands: [],
    groups: [],
    current_help_heading: None,
    current_disp_ord: Some(
        0,
    ),
    subcommand_value_name: None,
    subcommand_heading: None,
    external_value_parser: None,
    long_help_exists: false,
    deferred: None,
    app_ext: Extensions {
        extensions: FlatMap {

## arg

In [ ]:
nominal arg = $'clap_Arg'

()



## new_arg

In [ ]:
inl new_arg (s : rust.static_ref' sm'.str) : arg =
    !\\(s, $'"clap::Arg::new($0)"')

()



In [ ]:
// // test
// // print_code=false
// // rust=

types ()
inl arg = ##"arg" |> new_arg
trace Debug
    fun () => $'$"new_arg"'
    fun () =>
        inl arg = arg |> sm'.format_pretty'
        $'$"arg: {!arg} / {!_locals ()}"'

11:07:39 debug   #1 new_arg / arg: Arg {
    id: "arg",
    help: None,
    long_help: None,
    action: None,
    value_parser: None,
    blacklist: [],
    settings: ArgFlags(
        0,
    ),
    overrides: [],
    groups: [],
    requires: [],
    r_ifs: [],
    r_unless: [],
    short: None,
    long: None,
    aliases: [],
    short_aliases: [],
    disp_ord: None,
    val_names: [],
    num_vals: None,
    val_delim: None,
    default_vals: [],
    default_vals_ifs: [],
    terminator: None,
    index: None,
    help_heading: None,
    value_hint: None,
    default_missing_vals: [],
}


## command_arg

In [ ]:
inl command_arg (arg : arg) (command : command) : command =
    !\\((command, arg), $'"clap::Command::arg($0, $1)"')

()



## arg_required

In [ ]:
inl arg_required (value : bool) (arg : arg) : arg =
    !\\((arg, value), $'"$0.required($1)"')

()



## arg_short

In [ ]:
inl arg_short (value : char) (arg : arg) : arg =
    !\\((arg, value), $'"$0.short($1)"')

()



## arg_long

In [ ]:
inl arg_long (value : rust.static_ref' sm'.str) (arg : arg) : arg =
    !\\((arg, value), $'"$0.long($1)"')

()



## arg_index

In [ ]:
inl arg_index (value : i32) (arg : arg) : arg =
    !\\((arg, value), $'"$0.index($1)"')

()



## arg_index

In [ ]:
inl arg_index (value : i32) (arg : arg) : arg =
    !\\((arg, value), $'"$0.index($1)"')

()



## arg_matches

In [ ]:
nominal arg_matches = $'clap_ArgMatches'

()



## command_get_matches

In [ ]:
inl command_get_matches (command : command) : arg_matches =
    !\\(command, $'"clap::Command::get_matches($0)"')

()



## matches_get_one

In [ ]:
inl matches_get_one (x : string) (matches : arg_matches) : optionm'.option' sm'.std_string =
    inl x = join x
    inl x = x |> sm'.as_str
    !\($'"clap::ArgMatches::get_one(&!matches, !x).cloned()"')

()



## command_debug_assert

In [ ]:
inl command_debug_assert (command : command) : () =
    !\\(command, $'"clap::Command::debug_assert($0)"')

()



## main

In [ ]:
inl main () =
    types ()
    init_trace_state ()
    $"let is_windows () = !is_windows ()" : ()
    $"let get_executable_suffix () = !get_executable_suffix ()" : ()
    $"let execute_async x = !execute_async x" : ()
    $"let execute_with_options_async x = !execute_with_options_async x" : ()
    $"let split_args x = !split_args x" : ()

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>]